## 케라스 모델과 레이어를 맞춤형으로 사용하기
##### url: https://www.tensorflow.org/guide/keras/custom_layers_and_models

- 목차
    - 설정
    - 레이어 클래스
        - 레이어에서 가중치와 연산 캡슐화하기
        - 모범 예제: 입력 차원이 알려지기 전까지 가중치 생성 지연시키기
        - 재귀적으로 레이어 구성하기
        - 데이터가 모델을 통과하는 동안 생성되는 손실 값을 재귀적으로 수집하는 레이어
        - 선택적으로 레이어 직렬화 활성화하기
        - Call 메소드가 가지는 훈련 인자에 대한 특권
    - 모델 작성하기
        - 모델 클래스
        - 하나로 합치기: 엔드 투 엔드 예제
        - 객체 지향 개발 기법을 넘어서: 함수형 API
        
### 설정

In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf



tf.keras.backend.clear_session() # 노트북의 상태를 쉽게 초기화할 수 있습니다.

## 레이어 클래스
### 레이어에서 가중치와 연산 캡슐화하기
사용하게 될 주요 데이터구조는 `레이어` 입니다. 레이어는 상태(레이어의 가중치,'weight')와 입력을 출력으로 변환하는 것(호출,'call', 레이어의 순전파)을 캡슐화합니다.<br><br>
여기 밀집되어 연결된 레이어가 있습니다. 이것들은 변수 `w`와 `b`로 나타내어지는 상태를 가지고 있습니다. 

In [2]:
from tensorflow.keras import layers

class Linear(layers.Layer):
    
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(initial_value=w_init(shape=(input_dim, units),
                                                  dtype='float32'),
                             trainable=True)
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(initial_value=b_init(shape=(units,),
                                                  dtype='float32'),
                             trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

x = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[-0.12304512  0.06049802 -0.07464755 -0.04367318]
 [-0.12304512  0.06049802 -0.07464755 -0.04367318]], shape=(2, 4), dtype=float32)


`w`와 `b`가 자동적으로 레이어 속성 집합으로써 추적된다는 것을 확인해보세요.

In [3]:
assert linear_layer.weights == [linear_layer.w, linear_layer.b]

또한, `add_weight` 메소드를 사용하여 레이어가 가진 가중치에 더 빠르게 접근할 수 있습니다.

In [4]:
class Linear(layers.Layer):
    
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                initializer='random_normal',
                                trainable=True)
        self.b = self.add_weight(shape=(units,),
                                initializer='zeros',
                                trainable=True)
        
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b
    
s = tf.ones((2, 2))
linear_layer = Linear(4, 2)
y = linear_layer(x)
print(y)

tf.Tensor(
[[ 0.04412395  0.03380433 -0.01357309  0.04148406]
 [ 0.04412395  0.03380433 -0.01357309  0.04148406]], shape=(2, 4), dtype=float32)


#### 레이어는 학습할 수 없는 가중치를 가질 수 있습니다. 
학습 가능한 가중치들 사이에서, 레이어에 학습할 수 없는 가중치들을 추가할 수 있습니다.<br>
레이어를 학습하는 과정에서 역전파 시 이러한 가중치들은 고려하지 않아야 합니다.<br><br>
어떻게 학습할 수 없는 가중치를 더하고 사용하는지 살펴보겠습니다.

In [5]:
class ComputeSum(layers.Layer):
    
    def __init__(self, input_dim):
        super(ComputeSum, self).__init__()
        self.total = tf.Variable(initial_value=tf.zeros((input_dim,)),
                                 trainable=False)
        
    def call(self, inputs):
        self.total.assign_add(tf.reduce_sum(inputs, axis=0))
        return self.total
    
x = tf.ones((2, 2))
my_sum = ComputeSum(2)
y = my_sum(x)
print(y.numpy())
y = my_sum(x)
print(y.numpy())

[2. 2.]
[4. 4.]


해당 가중치들은 `layer.weights`의 일부분이지만, 학습할 수 없는 가중치로 분류됩니다.

In [6]:
print('weights:', len(my_sum.weights))
print('non-trainable weights:', len(my_sum.non_trainable_weights))

# 학습할 수 있는 가중치가 포함되어 있지 않습니다.
print('trainable_weights:', my_sum.trainable_weights)

weights: 1
non-trainable weights: 1
trainable_weights: []


### 모범 예제: 입력 차원이 알려지기 전까지 가중치 생성 지연시키기

로지스틱 회귀 예시에서, `Linear` 레이어는 `__init__` 메소드에서 가중치 `w`와 `b`의 크기를 계산하기 위해 사용되는 `input_dim`을 인자로 가지고 있습니다.

In [7]:
class Linear(layers. Layer):
    
    def __init__(self, units=32, input_dim=32):
        super(Linear, self).__init__()
        self.w = self.add_weight(shape=(input_dim, units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(units, ),
                                 initializer='zeros',
                                 trainable=True)

많은 경우에서 입력의 크기를 미리 알 수 없습니다. 또한 레이어를 먼저 생성한 후, 입력 값이 알려졌을 때 게으르게(lazily) 가중치를 생성하길 원할 수도 있습니다.<br><br>
Keras API에서, 레이어의 가중치를 `build(input_shape)` 메소드에서 생성하는 것을 권합니다. 해당 메소드는 다음과 같습니다. 

In [8]:
class Linear(layers.Layer):
    
    def __init__(self, units=32):
        super(Linear, self).__init__()
        self.units = units
        
    def build(self, input_shape):
        self.w = self.add_weight(shape=(input_shape[-1], self.units),
                                 initializer='random_normal',
                                 trainable=True)
        self.b = self.add_weight(shape=(self.units, ),
                                 initializer='random_normal',
                                 trainable=True)
    
    def call(self, inputs):
        return tf.matmul(inputs, self.w) + self.b

레이어의 `__call__` 메소드는 자동적으로 `build` 메소드를 가장 먼저 호출하여 수행합니다. 이제 게으르고(lazy) 사용하기 쉬운 레이어가 완성되었습니다.

In [9]:
linear_layer = Linear(32) # 인스턴스를 생성하는 단계에서, 호출될 입력 데이터에 대해 알지 못합니다.
y = linear_layer(x) # 레이어의 가중치는 해당 레이어가 호출되었을 때 가장 먼저 동적으로 생성됩니다.

### 재귀적으로 레이어 구성하기
만약, 레이어 인스턴스를 다른 레이어의 속성으로서 할당하게되면, 외부 레이어는 내부 레이어의 가중치를 트래킹하기 시작합니다.<br><br>
`__init__` 메소드 내에서 이러한 서브레이어를 생성하는 것이 좋습니다. (서브레이어들은 일반적으로 `build` 메소드를 가지고 있으므로 외부 레이어가 생성(build)될 때 해당 레이어들도 같이 생성됩니다.)

In [10]:
# Linear 클래스를 재사용한다고 가정해봅시다.
# `build` 메소드는 위에서 정의한대로 사용합니다.

class MLPBlock(layers.Layer):
    
    def __init__(self):
        super(MLPBlock, self).__init__()
        self.linear_1 = Linear(32)
        self.linear_2 = Linear(32)
        self.linear_3 = Linear(1)
        
    def call(self, inputs):
        x = self.linear_1(inputs)
        x = tf.nn.relu(x)
        x = self.linear_2(x)
        x = tf.nn.relu(x)
        return self.linear_3(x)
    

mlp = MLPBlock()
y = mlp(tf.ones(shape=(3, 64)))
print('weights:', len(mlp.weights))
print('trainable weights:', len(mlp.trainable_weights))

weights: 6
trainable weights: 6


### 데이터가 모델을 통과하는 동안 생성되는 손실 값을 재귀적으로 수집하는 레이어
`call` 메소드를 작성할 때, 나중에 학습 루프를 작성할 때 사용할 손실 텐서를 만들 수 있습니다.<br>
이는 `self.add_loss(value)`를 호출해서 만들 수 있습니다.

In [11]:
# 활성화 정규화 손실 함수를 생성하는 레이어
class ActivityRegularizationLayer(layers.Layer):
    
    def __init__(self, rate=1e-2):
        super(ActivityRegularizationLayer, self).__init__()
        self.rate = rate
        
    def call(self, inputs):
        self.add_loss(self.rate * tf.reduce_sum(inputs))
        return inputs

이러한 손실 값들은(내부 레이어 어디서든 생성되는 것들을 포함하여) `layer.losses`를 통해 값을 확인할 수 있습니다.<br>
해당 값은 맨 앞 레이어의 `__call__`이 호출될 때마다 초기화 됩니다. 그래서 `layer.losses`는 마지막 학습에서 생성된 손실 값을 가지고 있게 됩니다.

In [12]:
class OuterLayer(layers.Layer):
    
    def __init__(self):
        super(OuterLayer, self).__init__()
        self.activity_reg = ActivityRegularizationLayer(1e-2)
        
    def call(self, inputs):
        return self.activity_reg(inputs)
    
layer = OuterLayer()
assert len(layer.losses) == 0 # 레이어가 아직 호출되지 않았기 때문에 손실 값이 존재하지 않습니다.
_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1 # 레이어를 호출함으로써 손실 값 하나를 생성했습니다.

# `layer.losses` 는 각 __call__ 메소드의 시작에서 초기화됩니다.
_ = layer(tf.zeros(1, 1))
assert len(layer.losses) == 1 # 현재 손실 값은 마지막 호출에서 생성된 값 입니다.

추가적으로 `loss`는 내부 레이어의 가중치에서 생성된 손실 값도 포함하고 있습니다.

In [13]:
class OuterLayer(layers.Layer):
    
    def __init__(self):
        super(OuterLayer, self).__init__()
        self.dense = layers.Dense(32, kernel_regularizer = tf.keras.regularizers.l2(1e-3))
        
    def call(self, inputs):
        return self.dense(inputs)
    
    
layer = OuterLayer()
_ = layer(tf.zeros((1, 1)))

# 위의 'kernel_regularizaer'에서 생성된 값은 
# 1e-3 * sum(layer.dense.kernel ** 2)와 같습니다.
print(layer.losses)

[<tf.Tensor: shape=(), dtype=float32, numpy=0.001842881>]


이러한 손실 값들은 학습 루프를 작성할 때, 다음과 같이 고려됩니다.

In [ ]:
# 옵티마이저를 생성 합니다.
optimizer = tf.keras.optimizers.SGD(learning_rate=1e-3)
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

# 데이터셋의 배치를 사용해 반복합니다.
for x_batch_train, y_batch_train in train_dataset:
    with tf.GradientTape() as tape:
        logits = layer(x_batch_train) # 미니 배치의 로짓
        loss_value = loss_fn(y_batch_train, logits) # 미니 배치의 손실 값
        loss_value += sum(model.losses) # 순전파가 진행되는 동안 생성된 추가 로스 더하기
        
    grads = tape.gradient(loss_Value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

학습 루프에 대한 자세한 가이드는 두 번째 섹션의 `guide to training and evaluation`에서 다룹니다.